In [1]:
pip install pygame --pre

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pydub --pre

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pygame, sys, random, itertools, csv, os, math, json
import pandas as pd
dirname = os.path.dirname(os.path.abspath("__file__"))
#print(dirname)
import random
from pydub.generators import Sine
import time

import numpy as np
import scipy.io.wavfile as wavfile
from pydub import AudioSegment
from pygame import mixer

from PyQt5.QtWidgets import QApplication, QProgressDialog

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\user\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [16]:

# --- constants -----------------------------------------------------------------------
import subprocess


WIDTH = 1366
HEIGHT = 768

# กำหนดสี
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)

FPS = 1000
current_time = 0
choice_number = 1

id_student = ''
csv_columns = ['Subject ID', 'Block', 'No.', 'Picture', 'Sound', 'Result', 'Time']
#print(csv_file)

duration_time = 0
dict_data = []
Group_type = 'Group1'
Block = 1
check_num = 0 #จำนวนวงกลม



double = 0
Set = 0
Set1 = 6
add_num = 0

# กำหนดสีและน้ำหนัก
available_colors = [
    (pygame.Color(255, 0, 0), 1),   # สีแดง (น้ำหนัก 1)
    (pygame.Color(0, 255, 0), 2),   # สีเขียว (น้ำหนัก 2)
]
selected_color = None

#selected_color = random.choice(available_colors)

class Button(object):

    def __init__(self, position, size, color, text):

        self.image = pygame.Surface(size)
        self.image.fill(color)
        self.rect = pygame.Rect((0,0), size)
        self.color = color
        self.rect.topleft = position

    def draw(self, screen):
        screen.blit(self.image, self.rect)
        pygame.draw.rect(screen, self.color, self.rect, border_radius=5)

    def is_clicked(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                return self.rect.collidepoint(pygame.mouse.get_pos())

class Circle(object):

    def __init__(self, position, size, color, text):

        self.image = pygame.Surface(size)
        self.image.fill(color)
        self.rect = position

    def draw(self, screen):
        #screen.blit(self.image, self.rect)
        #selected_color = random.choice(available_colors)
        color = selected_color
        pygame.draw.circle(screen, color, self.rect, 100)
        #screen.fill((0, 0, 0))

    def is_clicked(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                return self.rect.collidepoint(pygame.mouse.get_pos())
            
def writecsv():
    global id_student, dict_data, Group_type
    try:
        csv_file = os.path.join(dirname, 'result/'+str(id_student)+'.csv')
        with open(csv_file, 'w', newline='') as csvfile:
            writera = csv.DictWriter(csvfile, fieldnames= csv_columns)
            writera.writeheader()
                #writer = csv.DictWriter(csvfile, fieldnames= csv_columns)
            for data in dict_data:
                writera.writerow(data)
            csvfile.close()
        df = pd.read_csv(csv_file)
        group = df.groupby(['Result'])['Time'].mean()
        count_result = df.groupby(['Result'])['No.'].count()
        count_correct = count_result['correct'] if 'correct' in count_result.keys() else 0
        count_miss = count_result['miss'] if 'miss' in count_result.keys() else 0
        count_wrong = count_result['wrong'] if 'wrong' in count_result.keys() else 0
        summary_data = {'count_correct': str(count_correct), 'count_wrong': str(count_wrong), 'count_miss': str(count_miss), 'avg_correct': '%.3f' %(group['correct'] if 'correct' in count_result.keys() else 0)}
        print(summary_data)
        with open(csv_file, 'a', newline='\n') as csvfile:
            writera = csv.DictWriter(csvfile, fieldnames= ['count_correct', 'count_wrong', 'count_miss', 'avg_correct'])
            writera.writeheader()
                #writer = csv.DictWriter(csvfile, fieldnames= csv_columns)
            writera.writerow(summary_data)
            csvfile.close()
        dict_data.clear()
        print("Succesfully")
        
    except Exception as e:
        print(e)
        
#--------------------------------------------------------------------------------------------------------
def generate_tone_with_silence(frequency, on_duration_ms, off_duration_ms, total_duration_ms, volume=0.5):
    # สร้างเสียง sine wave ที่ความถี่ที่กำหนด
    tone = Sine(frequency, sample_rate=44100)
    
    # สร้างเซ็กเมนต์เสียงดัง
    sound = tone.to_audio_segment(duration=on_duration_ms).apply_gain(20 * np.log10(volume))
    # สร้างเซ็กเมนต์เสียงเงียบ
    silence = AudioSegment.silent(duration=off_duration_ms)

    # ต่อเสียงดังและเสียงเงียบเป็นลูป
    cycle = sound + silence
    cycles = total_duration_ms // (on_duration_ms + off_duration_ms)
    result = cycle * cycles

    # ตรวจสอบระยะเวลาสุดท้ายของไฟล์เสียง
    final_cycle_duration_ms = total_duration_ms % (on_duration_ms + off_duration_ms)
    if final_cycle_duration_ms > on_duration_ms:
        result += sound + AudioSegment.silent(duration=(final_cycle_duration_ms - on_duration_ms))
    elif final_cycle_duration_ms > 0:
        result += tone.to_audio_segment(duration=final_cycle_duration_ms)

    return result

def save_sound(directory, file_name, frequency, on_duration, off_duration, total_duration, volume=0.5):
    sound = generate_tone_with_silence(frequency, on_duration, off_duration, total_duration, volume)
    file_path = os.path.join(directory, file_name)
    sound.export(file_path, format="wav")
    print(f"เสียงถูกบันทึกเป็นไฟล์ {file_name}")
#--------------------------------------------------------------------------------------------------------
def stage1(screen): #login
    try:
        clock = pygame.time.Clock()
        global id_student, duration_time, Group_type, add_num,Block 
        pygame.display.update()
        # กำหนดสี
        WHITE = (255, 255, 255)
        BLACK = (0, 0, 0)
        GRAY = (200, 200, 200)
        BACKGROUND_COLOR = BLACK
        data = load_data()
        data1 = load_data1()
        # กำหนดตำแหน่งของ Input Box
        USERNAME_POS = (WIDTH // 4, HEIGHT // 2.5)
        CELL_TYPE_POS = (WIDTH // 1.7, HEIGHT // 2.5)
        TEXTBOX_WIDTH, TEXTBOX_HEIGHT = 500, 80
        COMBOBOX_WIDTH, COMBOBOX_HEIGHT = 200, 80
        ARROW_WIDTH, ARROW_HEIGHT = 20, 15

        # กำหนดตำแหน่งของปุ่ม Login
        LOGIN_BUTTON_POS = (WIDTH // 2 - 50, HEIGHT // 1.8)
        BUTTON_WIDTH, BUTTON_HEIGHT = 100, 40

        # สร้าง font
        font = pygame.font.Font(None, 36)

        # สร้าง Input Box
        username_rect = pygame.Rect(USERNAME_POS, (TEXTBOX_WIDTH, TEXTBOX_HEIGHT))
        cell_type_rect = pygame.Rect(CELL_TYPE_POS, (COMBOBOX_WIDTH, COMBOBOX_HEIGHT))
        textbox_color = GRAY
        username_active = False
        cell_type_active = False
        username_input = ""
        cell_type_options = ["Group1", "Group2"]
        selected_cell_type_index = 0

        # สร้างปุ่ม Login
        login_button_rect = pygame.Rect(LOGIN_BUTTON_POS, (BUTTON_WIDTH, BUTTON_HEIGHT))
        login_button_color = (0, 150, 0)
    
        # สร้างลูกศร
        arrow_rect = pygame.Rect(CELL_TYPE_POS[0] + COMBOBOX_WIDTH - ARROW_WIDTH, CELL_TYPE_POS[1], ARROW_WIDTH, ARROW_HEIGHT)
        arrow_color = BLACK

        def draw_text(text, font, color, surface, x, y):
            textobj = font.render(text, 1, color)
            textrect = textobj.get_rect()
            textrect.topleft = (x, y)
            surface.blit(textobj, textrect)

        while True:
            ticks = (pygame.time.get_ticks())
            duration_time = ticks
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_pos = pygame.mouse.get_pos()

                    # ตรวจสอบการคลิก Input Box
                    if username_rect.collidepoint(mouse_pos):
                        username_active = not username_active
                        cell_type_active = False
                    elif cell_type_rect.collidepoint(mouse_pos):
                        cell_type_active = not cell_type_active
                        username_active = False
                    else:
                        username_active = False
                        cell_type_active = False

                    # ตรวจสอบการคลิกปุ่ม Login
                    if login_button_rect.collidepoint(mouse_pos):
                        print("Login Clicked")
                        print("Username:", id_student)
                        print("Cell Type:", cell_type_options[selected_cell_type_index])
                        Group_type = cell_type_options[selected_cell_type_index]
                        
                        if Group_type == 'Group1':
                            Block = 1
                        else  :
                            Block = 2
                            
                        if id_student == "admin":
                            admin1(screen)
                        else:
                            stage2(screen)
                       
                        # ทำงานที่ต้องการเมื่อคลิกปุ่ม Login, เช่น ตรวจสอบข้อมูลผู้ใช้

                    # ตรวจสอบการคลิกลูกศร
                    if arrow_rect.collidepoint(mouse_pos):
                        cell_type_active = not cell_type_active

                    # ตรวจสอบการคลิกตัวเลือกทางลิ้นชัก
                    for i in range(len(cell_type_options)):
                        option_rect = pygame.Rect(CELL_TYPE_POS[0], CELL_TYPE_POS[1] + COMBOBOX_HEIGHT * (i + 1), COMBOBOX_WIDTH, COMBOBOX_HEIGHT)
                        if option_rect.collidepoint(mouse_pos):
                            selected_cell_type_index = i
                            cell_type_active = False

                elif event.type == pygame.KEYDOWN:
                    if username_active:
                        if event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            sys.exit()
                        elif event.key == pygame.K_BACKSPACE:
                            id_student = id_student[:-1]
                        elif event.key == pygame.K_RETURN:
                            cell_type_active = True
                            if id_student == "admin":
                                admin1(screen)
                            else:
                                stage2(screen)
                       
                        else:
                            id_student += event.unicode

                    elif cell_type_active:
                        if event.key == pygame.K_RETURN:
                             cell_type_active = False
                        elif event.key == pygame.K_UP:
                            selected_cell_type_index = (selected_cell_type_index - 1) % len(cell_type_options)
                            if id_student == "admin":
                                admin1(screen)
                            else:
                                stage2(screen)
                       
                        elif event.key == pygame.K_DOWN:
                            selected_cell_type_index = (selected_cell_type_index + 1) % len(cell_type_options)

            # ล้างหน้าจอ
            screen.fill(BACKGROUND_COLOR)

            # วาด Input Box
            pygame.draw.rect(screen, textbox_color if not username_active else WHITE, username_rect)
            pygame.draw.rect(screen, textbox_color if not cell_type_active else GRAY, cell_type_rect)
            draw_text(id_student, font,BLACK, screen, USERNAME_POS[0] + 5, USERNAME_POS[1] + 5)
            draw_text(cell_type_options[selected_cell_type_index], font, WHITE, screen, CELL_TYPE_POS[0] + 5, CELL_TYPE_POS[1] + 5)


            # วาดปุ่ม Login
            pygame.draw.rect(screen, login_button_color, login_button_rect)
            draw_text("Login", font, WHITE, screen, LOGIN_BUTTON_POS[0] + 25, LOGIN_BUTTON_POS[1] + 10)

            # วาดลูกศร
            pygame.draw.polygon(screen, arrow_color, [(arrow_rect.x, arrow_rect.y),
                                                      (arrow_rect.x + ARROW_WIDTH, arrow_rect.y),
                                                      (arrow_rect.x + ARROW_WIDTH / 2, arrow_rect.y + ARROW_HEIGHT)])

            # วาดตัวเลือกทางลิ้นชัก
            if cell_type_active:
                for i, option in enumerate(cell_type_options):
                    option_rect = pygame.Rect(CELL_TYPE_POS[0], CELL_TYPE_POS[1] + COMBOBOX_HEIGHT * (i + 1), COMBOBOX_WIDTH, COMBOBOX_HEIGHT)
                    pygame.draw.rect(screen, WHITE, option_rect, 2)
                    draw_text(option, font, WHITE, screen, CELL_TYPE_POS[0] + 5, option_rect.y + 5)

            # อัพเดทหน้าจอ
            pygame.display.flip()
            clock.tick(FPS)

        # กดปุ่ม window ออก
    except Exception as e:
        print(e)
#-----------------------------------------------------------------------------
class TextBox:
    def __init__(self, rect, text='', font=None, text_color=BLACK, bg_color=GRAY):
        self.rect = pygame.Rect(rect)
        self.text = text
        self.font = font or pygame.font.Font(None, self.rect.height - 4)
        self.text_color = text_color
        self.bg_color = bg_color
        self.render_text()

    def render_text(self):
        self.rendered_text = self.font.render(self.text, True, self.text_color)
        self.rendered_text_rect = self.rendered_text.get_rect(x=self.rect.x + 2, centery=self.rect.centery)

    def draw(self, surface):
        pygame.draw.rect(surface, self.bg_color, self.rect)
        surface.blit(self.rendered_text, self.rendered_text_rect)

    def update(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                self.editing = True
            else:
                self.editing = False
        if event.type == pygame.KEYDOWN and self.editing:
            if event.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                self.text += event.unicode
            self.render_text()

def save_data(data):
    with open('data.json', 'w') as file:
        json.dump(data, file)

def load_data():
    if os.path.exists('data.json'):
        with open('data.json', 'r') as file:
            return json.load(file)
    else:
        return {}
    
# สร้างฟังก์ชันสำหรับสร้างหน้าต่างแจ้งเตือน
def show_loading_dialog(screen):
    # กำหนดขนาดและตำแหน่งของหน้าต่าง
    dialog_width = 300
    dialog_height = 100
    dialog_x = (screen.get_width() - dialog_width) // 2
    dialog_y = (screen.get_height() - dialog_height) // 2

    # สร้างพื้นหลังสีขาว
    dialog_surface = pygame.Surface((dialog_width, dialog_height))
    dialog_surface.fill((255, 255, 255))

    # สร้างข้อความ
    font = pygame.font.Font(None, 24)
    text_surface = font.render("Loading...", True, (0, 0, 0))
    text_rect = text_surface.get_rect(center=(dialog_width // 2, dialog_height // 2))

    # วาดหน้าต่างแจ้งเตือน
    dialog_surface.blit(text_surface, text_rect)
    screen.blit(dialog_surface, (dialog_x, dialog_y))
    pygame.display.flip()
    
def admin1(screen):
    try:
        clock = pygame.time.Clock()
        global id_student, duration_time
        
        # กำหนดตำแหน่งของปุ่ม Login
        BACK_BUTTON_POS = (20, HEIGHT -60)
        NEXT_BUTTON_POS = (WIDTH - 250, HEIGHT - 60)
        SAVE_BUTTON_POS = (WIDTH - 120, HEIGHT - 60)
        
        
        BUTTON_WIDTH, BUTTON_HEIGHT = 100, 40
        
        # สร้างปุ่ม Login
        back_button_rect = pygame.Rect(BACK_BUTTON_POS, (BUTTON_WIDTH, BUTTON_HEIGHT))
        next_button_rect = pygame.Rect(NEXT_BUTTON_POS, (BUTTON_WIDTH, BUTTON_HEIGHT))
        save_button_rect = pygame.Rect(SAVE_BUTTON_POS, (BUTTON_WIDTH, BUTTON_HEIGHT))
        button_color = (0, 150, 0)
        
       # โหลดข้อมูลที่บันทึกไว้ (หากมี)
        data = load_data()
        # สร้าง TextBoxes
        S1 = TextBox((350, 130, 200, 40), text=data.get('S1', 'nosound'), font=None, text_color=BLACK, bg_color=GRAY)
        S2 = TextBox((350, 230, 200, 40), text=data.get('S2', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        S3 = TextBox((350, 330, 200, 40), text=data.get('S3', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        S4 = TextBox((350, 430, 200, 40), text=data.get('S4', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        S5 = TextBox((350, 530, 200, 40), text=data.get('S5', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        S6 = TextBox((350, 630, 200, 40), text=data.get('S6', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        S7 = TextBox((350, 730, 200, 40), text=data.get('S7', '40'), font=None, text_color=BLACK, bg_color=GRAY)

        p1 = TextBox((650, 130, 200, 40), text=data.get('p1', 'nosound'), font=None, text_color=BLACK, bg_color=GRAY)
        p2 = TextBox((650, 230, 200, 40), text=data.get('p2', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        p3 = TextBox((650, 330, 200, 40), text=data.get('p3', '80'), font=None, text_color=BLACK, bg_color=GRAY)
        p4 = TextBox((650, 430, 200, 40), text=data.get('p4', '120'), font=None, text_color=BLACK, bg_color=GRAY)
        p5 = TextBox((650, 530, 200, 40), text=data.get('p5', '160'), font=None, text_color=BLACK, bg_color=GRAY)
        p6 = TextBox((650, 630, 200, 40), text=data.get('p6', '200'), font=None, text_color=BLACK, bg_color=GRAY)
        p7 = TextBox((650, 730, 200, 40), text=data.get('p7', 'infin'), font=None, text_color=BLACK, bg_color=GRAY)

        num1 = TextBox((950, 130, 200, 40), text=data.get('num1', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num2 = TextBox((950, 230, 200, 40), text=data.get('num2', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num3 = TextBox((950, 330, 200, 40), text=data.get('num3', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num4 = TextBox((950, 430, 200, 40), text=data.get('num4', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num5 = TextBox((950, 530, 200, 40), text=data.get('num5', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num6 = TextBox((950, 630, 200, 40), text=data.get('num6', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num7 = TextBox((950, 730, 200, 40), text=data.get('num7', '5'), font=None, text_color=BLACK, bg_color=GRAY)

        text_boxes = [S1, S2, S3, S4, S5, S6, S7, p1, p2, p3, p4, p5, p6, p7, num1, num2, num3, num4, num5, num6, num7]
            
        # กำหนดแบบอักษรและขนาดของตัวอักษร
        font = pygame.font.Font(None, 36)

        # สร้างพื้นผิวข้อความ "goup1"
        text_surface = font.render('Block1', True, WHITE)
        text_sound = font.render('Sound', True, WHITE)
        text_Pause = font.render('Pause', True, WHITE)
        text_Num = font.render('Number', True, WHITE)

        def draw_text(text, font, color, surface, x, y):
            textobj = font.render(text, 1, color)
            textrect = textobj.get_rect()
            textrect.topleft = (x, y)
            surface.blit(textobj, textrect)

        while True:
            duration_textboxes = [(S2, p2), (S3, p3), (S4, p4), (S5, p5), (S6, p6), (S7, p7)]
            i = 40
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_pos = pygame.mouse.get_pos()

                    # ตรวจสอบการคลิกปุ่ม Login
                    if back_button_rect.collidepoint(mouse_pos):
                        stage1(screen)
                    if next_button_rect.collidepoint(mouse_pos):
                        # เก็บค่าที่ถูกแก้ไขล่าสุดลงในไฟล์
                        data = {'S1': S1.text, 'S2': S2.text, 'S3': S3.text, 'S4': S4.text, 'S5': S5.text, 'S6': S6.text, 'S7': S7.text, 'p1': p1.text, 'p2': p2.text, 'p3': p3.text, 'p4': p4.text, 'p5': p5.text, 'p6': p6.text, 'p7': p7.text, 'num1': num1.text, 'num2': num2.text, 'num3': num3.text, 'num4': num4.text, 'num5': num5.text, 'num6': num6.text, 'num7': num7.text}
                        save_data(data)
                        admin2(screen)
                    if save_button_rect.collidepoint(mouse_pos):
                        # เก็บค่าที่ถูกแก้ไขล่าสุดลงในไฟล์
                        data = {'S1': S1.text, 'S2': S2.text, 'S3': S3.text, 'S4': S4.text, 'S5': S5.text, 'S6': S6.text, 'S7': S7.text, 'p1': p1.text, 'p2': p2.text, 'p3': p3.text, 'p4': p4.text, 'p5': p5.text, 'p6': p6.text, 'p7': p7.text, 'num1': num1.text, 'num2': num2.text, 'num3': num3.text, 'num4': num4.text, 'num5': num5.text, 'num6': num6.text, 'num7': num7.text}
                        save_data(data)
                        # วนลูปผ่านทุก TextBox และส่งพารามิเตอร์ on_duration และ off_duration ไปยังฟังก์ชัน save_sound()
                        show_loading_dialog(screen)
                        for on_textbox, off_textbox in duration_textboxes:
                            directory = "tmp_sound"
                            file_name = f"sound_P{i}.wav"
                            i += 40
                            frequency = 1500
                            try:
                                on_value = int(on_textbox.text)
                            except ValueError:
                                on_value = 0
                            try:
                                off_value = int(off_textbox.text)
                            except ValueError:
                                off_value = 0 

                            total_value = 20000

                            # ส่งค่าตัวเลขไปยังฟังก์ชัน save_sound() ในรูปแบบของพารามิเตอร์ on_duration และ off_duration
                            save_sound(directory, file_name, frequency, on_value, off_value, total_value, volume=0.5)
                       

                        
                for textbox in text_boxes:
                    textbox.update(event)
                
                
            screen.fill(BLACK)
            
            for textbox in text_boxes:
                textbox.draw(screen)

            # วาดข้อความ "goup1"
            screen.blit(text_surface, (50, 50))
            screen.blit(text_sound, (400, 50))
            screen.blit(text_Pause, (700, 50))
            screen.blit(text_Num, (1000, 50))

            # วาดปุ่ม Login
            pygame.draw.rect(screen, button_color, back_button_rect)
            draw_text("Back", font, WHITE, screen, BACK_BUTTON_POS[0] + 25, BACK_BUTTON_POS[1] + 10)
            pygame.draw.rect(screen, button_color, save_button_rect)
            draw_text("Save", font, WHITE, screen, SAVE_BUTTON_POS[0] + 25, SAVE_BUTTON_POS[1] + 10)
            pygame.draw.rect(screen, button_color, next_button_rect)
            draw_text("Next", font, WHITE, screen, NEXT_BUTTON_POS[0] + 25, NEXT_BUTTON_POS[1] + 10)


            pygame.display.flip()
            clock.tick(FPS)

    except Exception as e:
        print(e)

#------------------------------------------------------------------------------        
def save_data1(data):
    with open('data1.json', 'w') as file:
        json.dump(data, file)

def load_data1():
    if os.path.exists('data1.json'):
        with open('data1.json', 'r') as file:
            return json.load(file)
    else:
        return {}

def admin2(screen):
    try:
        clock = pygame.time.Clock()
        global id_student, duration_time
        
        # กำหนดตำแหน่งของปุ่ม Login
        BACK_BUTTON_POS = (20, HEIGHT -60)
        NEXT_BUTTON_POS = (WIDTH - 250, HEIGHT - 60)
        SAVE_BUTTON_POS = (WIDTH - 120, HEIGHT - 60)
        
        
        BUTTON_WIDTH, BUTTON_HEIGHT = 100, 40
        
        # สร้างปุ่ม Login
        back_button_rect = pygame.Rect(BACK_BUTTON_POS, (BUTTON_WIDTH, BUTTON_HEIGHT))
        next_button_rect = pygame.Rect(NEXT_BUTTON_POS, (BUTTON_WIDTH, BUTTON_HEIGHT))
        save_button_rect = pygame.Rect(SAVE_BUTTON_POS, (BUTTON_WIDTH, BUTTON_HEIGHT))
        button_color = (0, 150, 0)
        
       # โหลดข้อมูลที่บันทึกไว้ (หากมี)
        data2 = load_data1()
        # สร้าง TextBoxes
        S1 = TextBox((350, 130, 200, 40), text=data2.get('S1', 'nosound'), font=None, text_color=BLACK, bg_color=GRAY)
        S2 = TextBox((350, 230, 200, 40), text=data2.get('S2', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        S3 = TextBox((350, 330, 200, 40), text=data2.get('S3', '80'), font=None, text_color=BLACK, bg_color=GRAY)
        S4 = TextBox((350, 430, 200, 40), text=data2.get('S4', '120'), font=None, text_color=BLACK, bg_color=GRAY)
        S5 = TextBox((350, 530, 200, 40), text=data2.get('S5', '160'), font=None, text_color=BLACK, bg_color=GRAY)
        S6 = TextBox((350, 630, 200, 40), text=data2.get('S6', '200'), font=None, text_color=BLACK, bg_color=GRAY)
        S7 = TextBox((350, 730, 200, 40), text=data2.get('S7', '240'), font=None, text_color=BLACK, bg_color=GRAY)

        p1 = TextBox((650, 130, 200, 40), text=data2.get('p1', 'nosound'), font=None, text_color=BLACK, bg_color=GRAY)
        p2 = TextBox((650, 230, 200, 40), text=data2.get('p2', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        p3 = TextBox((650, 330, 200, 40), text=data2.get('p3', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        p4 = TextBox((650, 430, 200, 40), text=data2.get('p4', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        p5 = TextBox((650, 530, 200, 40), text=data2.get('p5', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        p6 = TextBox((650, 630, 200, 40), text=data2.get('p6', '40'), font=None, text_color=BLACK, bg_color=GRAY)
        p7 = TextBox((650, 730, 200, 40), text=data2.get('p7', 'infin'), font=None, text_color=BLACK, bg_color=GRAY)

        num1 = TextBox((950, 130, 200, 40), text=data2.get('num1', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num2 = TextBox((950, 230, 200, 40), text=data2.get('num2', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num3 = TextBox((950, 330, 200, 40), text=data2.get('num3', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num4 = TextBox((950, 430, 200, 40), text=data2.get('num4', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num5 = TextBox((950, 530, 200, 40), text=data2.get('num5', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num6 = TextBox((950, 630, 200, 40), text=data2.get('num6', '5'), font=None, text_color=BLACK, bg_color=GRAY)
        num7 = TextBox((950, 730, 200, 40), text=data2.get('num7', '5'), font=None, text_color=BLACK, bg_color=GRAY)

        text_boxes1 = [S1, S2, S3, S4, S5, S6, S7, p1, p2, p3, p4, p5, p6, p7, num1, num2, num3, num4, num5, num6, num7]
            
        # กำหนดแบบอักษรและขนาดของตัวอักษร
        font = pygame.font.Font(None, 36)

        # สร้างพื้นผิวข้อความ "goup1"
        text_surface = font.render('Block2', True, WHITE)
        text_sound = font.render('Sound', True, WHITE)
        text_Pause = font.render('Pause', True, WHITE)
        text_Num = font.render('Number', True, WHITE)

        def draw_text(text, font, color, surface, x, y):
            textobj = font.render(text, 1, color)
            textrect = textobj.get_rect()
            textrect.topleft = (x, y)
            surface.blit(textobj, textrect)

        while True:
            duration_textboxes = [(S2, p2), (S3, p3), (S4, p4), (S5, p5), (S6, p6), (S7, p7)]
            i = 40
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_pos = pygame.mouse.get_pos()
                    
                    # ตรวจสอบการคลิกปุ่ม Login
                    if back_button_rect.collidepoint(mouse_pos):
                        data2 = {'S1': S1.text, 'S2': S2.text, 'S3': S3.text, 'S4': S4.text, 'S5': S5.text, 'S6': S6.text, 'S7': S7.text, 'p1': p1.text, 'p2': p2.text, 'p3': p3.text, 'p4': p4.text, 'p5': p5.text, 'p6': p6.text, 'p7': p7.text, 'num1': num1.text, 'num2': num2.text, 'num3': num3.text, 'num4': num4.text, 'num5': num5.text, 'num6': num6.text, 'num7': num7.text}
                        #save_data1(data2)
                        admin1(screen)
                    if next_button_rect.collidepoint(mouse_pos):
                        # เก็บค่าที่ถูกแก้ไขล่าสุดลงในไฟล์
                        data2 = {'S1': S1.text, 'S2': S2.text, 'S3': S3.text, 'S4': S4.text, 'S5': S5.text, 'S6': S6.text, 'S7': S7.text, 'p1': p1.text, 'p2': p2.text, 'p3': p3.text, 'p4': p4.text, 'p5': p5.text, 'p6': p6.text, 'p7': p7.text, 'num1': num1.text, 'num2': num2.text, 'num3': num3.text, 'num4': num4.text, 'num5': num5.text, 'num6': num6.text, 'num7': num7.text}
                        save_data1(data2)
                        stage1(screen)
                    if save_button_rect.collidepoint(mouse_pos):
                        # เก็บค่าที่ถูกแก้ไขล่าสุดลงในไฟล์
                        data2 = {'S1': S1.text, 'S2': S2.text, 'S3': S3.text, 'S4': S4.text, 'S5': S5.text, 'S6': S6.text, 'S7': S7.text, 'p1': p1.text, 'p2': p2.text, 'p3': p3.text, 'p4': p4.text, 'p5': p5.text, 'p6': p6.text, 'p7': p7.text, 'num1': num1.text, 'num2': num2.text, 'num3': num3.text, 'num4': num4.text, 'num5': num5.text, 'num6': num6.text, 'num7': num7.text}
                        save_data1(data2)
                        show_loading_dialog(screen)
                        # วนลูปผ่านทุก TextBox และส่งพารามิเตอร์ on_duration และ off_duration ไปยังฟังก์ชัน save_sound()
                        for on_textbox, off_textbox in duration_textboxes:
                            directory = "tmp_sound"
                            file_name = f"sound_S{i}.wav"
                            i += 40
                            frequency = 1500
                            try:
                                on_value = int(on_textbox.text)
                            except ValueError:
                                on_value = 0
                            try:
                                off_value = int(off_textbox.text)
                            except ValueError:
                                off_value = 0 
                                
                            total_value = 20000

                            # ส่งค่าตัวเลขไปยังฟังก์ชัน save_sound() ในรูปแบบของพารามิเตอร์ on_duration และ off_duration
                            save_sound(directory, file_name, frequency, on_value, off_value, total_value, volume=0.5)
                        
                            
                        
                for textbox in text_boxes1:
                    textbox.update(event)
                
                
            screen.fill(BLACK)
            
            for textbox in text_boxes1:
                textbox.draw(screen)

            # วาดข้อความ "goup1"
            screen.blit(text_surface, (50, 50))
            screen.blit(text_sound, (400, 50))
            screen.blit(text_Pause, (700, 50))
            screen.blit(text_Num, (1000, 50))

            # วาดปุ่ม Login
            pygame.draw.rect(screen, button_color, back_button_rect)
            draw_text("Back", font, WHITE, screen, BACK_BUTTON_POS[0] + 25, BACK_BUTTON_POS[1] + 10)
            pygame.draw.rect(screen, button_color, save_button_rect)
            draw_text("Save", font, WHITE, screen, SAVE_BUTTON_POS[0] + 25, SAVE_BUTTON_POS[1] + 10)
            pygame.draw.rect(screen, button_color, next_button_rect)
            draw_text("Home", font, WHITE, screen, NEXT_BUTTON_POS[0] + 25, NEXT_BUTTON_POS[1] + 10)

            pygame.display.flip()
            clock.tick(FPS)
            

    except Exception as e:
        print(e)
#-----------------------------------------------------------------------------
data = load_data()
data1 = load_data1()
values = [data.get('num1'), data.get('num2'), data.get('num3'), data.get('num4'), data.get('num5'), data.get('num6'), data.get('num7'),
    data1.get('num1'), data1.get('num2'), data1.get('num3'), data1.get('num4'), data1.get('num5'), data1.get('num6'), data1.get('num7')]
num_values = [int(value) for value in values]

values1 = [data.get('num1'), data.get('num2'), data.get('num3'), data.get('num4'), data.get('num5'), data.get('num6'), data.get('num7')]
num1_values = [int(value) for value in values1]

values2 = [data1.get('num1'), data1.get('num2'), data1.get('num3'), data1.get('num4'), data1.get('num5'), data1.get('num6'), data1.get('num7')]
num2_values = [int(value) for value in values2]

total = sum(num_values)
total1 = sum(num1_values)
total2 = sum(num2_values)

num = total+1

def stage2(screen): #approve
    try:
        global duration_time
        button1 = Button((150, 527), (400, 100), (65,105,225), u"approve".encode('utf8').decode())
        button2 = Button((800, 527), (400, 100), (65,105,225), u"exit".encode('utf8').decode())
        # - mainloop -

        clock = pygame.time.Clock()
        start_ticks = pygame.time.get_ticks()
        img = pygame.image.load(os.path.join(dirname, 'img/intro2.png'))
        botton_approve = pygame.image.load(os.path.join(dirname, 'img/approve.png'))
        botton_exit = pygame.image.load(os.path.join(dirname, 'img/exit.png'))
        #counter = 2
        running = True

        while running:

            # - events -
            #ticks = (pygame.time.get_ticks() - start_ticks)/1000
            ticks = (pygame.time.get_ticks())
            #print('stage2: ', ticks)
            #duration_time = ticks
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    #writecsv()
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        #writecsv()
                        pygame.quit()
                        sys.exit()
                elif button1.is_clicked(event):
                    stage3(screen)

                elif button2.is_clicked(event):
                    #print("duration_time: ", duration_time)
                    pygame.quit()
                    sys.exit()

            # - draws -

            screen.fill((105,105,105))
            button1.draw(screen)
            button2.draw(screen)
            screen.blit(img,(0,150))
            screen.blit(botton_approve,(152,529))
            screen.blit(botton_exit,(802,529))
            pygame.display.flip()

            # - FPS -
            #round_play += 1
            clock.tick(FPS)
        
    except Exception as e:
        print(e)
        
def stage3(screen): #start
    try:
        # - mainloop -
        global duration_time
        #text = u"Break"
        button1 = Button((493, 538), (400, 100), (65,105,225), u"home".encode('utf8').decode())
        img = pygame.image.load(os.path.join(dirname, 'img/break3.png'))
        botton_con = pygame.image.load(os.path.join(dirname, 'img/continue5.png'))
        font = pygame.font.SysFont('Sans Serif', 40)
        clock = pygame.time.Clock()
        #counter = 2
        running = True

        while running:
            start_ticks = pygame.time.get_ticks()/1000
            # - events -
            #ticks = (pygame.time.get_ticks() - start_ticks)/1000
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    #print(dict_data)
                    writecsv()
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        writecsv()
                        pygame.quit()
                        sys.exit()
                if button1.is_clicked(event):
                    # go to stage3
                    duration_time = start_ticks
                    print("new duration time: ", duration_time)
                    stage4(screen)

            # - draws -

            screen.fill((0,0,0))
            #screen.blit(font.render(text, True, (0, 0, 0)), (325, 215))
            button1.draw(screen)
            img_rect = img.get_rect(center=screen.get_rect().center)
            screen.blit(img,(150,70))
            screen.blit(botton_con,(495,540))
            pygame.display.flip()

            # - FPS -
            #round_play += 1
            clock.tick(FPS)
        
    except Exception as e:
        print(e)
#----------------------------------------------------------------------------------------------------------------------------
pygame.mixer.init()
current_sound_index = 0
block1_dup = []
random_dup_label = []
sound =""
current_sound1_index = 0
block2_dup = []
random_dup1_label = []

def init_soundp():
    block1 = ['Nosound']
    n = 40
    for i in range(1, 7):
        file_path = f'tmp_sound/sound_P{n}.wav'
        block1.append(file_path)
        n += 40

    global block1_dup, random_dup_label
    block1_dup = block1 * 5
    random_dup_label = np.random.permutation(len(block1_dup))

    print("After permutation -->", random_dup_label)

# Function to play the next music file in the shuffled list
def playP_next_music():
    global current_sound_index,sound,current_sound1_index,random_dup1_label
    if current_sound_index < len(random_dup_label):
        sound_file = block1_dup[random_dup_label[current_sound_index]]
        current_sound_index += 1
        if sound_file != 'Nosound':
            pygame.mixer.music.stop()  # Stop the current music if it's playing
            pygame.mixer.music.load(sound_file)  # Load the new music file
            pygame.mixer.music.play()  # Play the new music
            if sound_file == "tmp_sound/sound_P240.wav":
                sound = "Sound_PCount"
            else:
                sound = file_name = os.path.splitext(os.path.basename(sound_file))[0]
            print(f"Now playing: {sound_file}") 
        else:
            sound = "P No sound"
            print("No sound to play")
    else: 
        return


#================================================
def init_sounds():
    block2 = ['Nosound']
    n = 40
    for i in range(1, 7):
        file_path = f'tmp_sound/sound_S{n}.wav'
        block2.append(file_path)
        n += 40

    global block2_dup, random_dup1_label
    block2_dup = block2 * 5
    random_dup1_label = np.random.permutation(len(block2_dup))

    print("After permutation -->", random_dup1_label)

# Function to play the next music file in the shuffled list
def playS_next_music():
    global current_sound1_index,sound,current_sound_index,random_dup_label
    if current_sound1_index < len(random_dup1_label):
        sound_file = block2_dup[random_dup1_label[current_sound1_index]]
        current_sound1_index += 1
        if sound_file != 'Nosound':
            pygame.mixer.music.stop()  # Stop the current music if it's playing
            pygame.mixer.music.load(sound_file)  # Load the new music file
            pygame.mixer.music.play()  # Play the new music
            if sound_file == "tmp_sound/sound_S240.wav":
                sound = "Sound_SCount"
            else:
                sound = file_name = os.path.splitext(os.path.basename(sound_file))[0]
            print(f"Now playing: {sound_file}") 
        else:
            sound = "S No sound"
            print("No sound to play")
    else:
        return
    
#==================================================
current_sound_p_index = 0
block_p_dup = []
random_dup_p_label = []
current_sound_s_index = 0
block_s_dup = []
random_dup_s_label = []
sound = ""

def init_sound_p():
    block_p = ['Nosound']
    n = 40
    for i in range(1, 7):
        file_path = f'tmp_sound/sound_P{n}.wav'
        block_p.append(file_path)
        n += 40

    global block_p_dup, random_dup_p_label
    block_p_dup = block_p * 2
    random_dup_p_label = np.random.permutation(len(block_p_dup))

    print("After permutation redp -->", random_dup_p_label)

# Function to play the next music file in the shuffled list
def play_p_next_music():
    global current_sound_p_index, sound, random_dup_p_label
    if current_sound_p_index < len(random_dup_p_label):
        sound_file = block_p_dup[random_dup_p_label[current_sound_p_index]]
        current_sound_p_index += 1
        if sound_file != 'Nosound':
            pygame.mixer.music.stop()  # Stop the current music if it's playing
            pygame.mixer.music.load(sound_file)  # Load the new music file
            pygame.mixer.music.play()  # Play the new music
            if sound_file == "tmp_sound/sound_P240.wav":
                sound = "Sound_PCount"
            else:
                sound = os.path.splitext(os.path.basename(sound_file))[0]
            print(f"Now playing red: {sound_file}")
        else:
            sound = "P No sound"
            print("No sound to play red")
    else:
        return
#==================================================
def init_sound_s():
    block_s = ['Nosound']
    n = 40
    for i in range(1, 7):
        file_path = f'tmp_sound/sound_S{n}.wav'
        block_s.append(file_path)
        n += 40

    global block_s_dup, random_dup_s_label
    block_s_dup = block_s * 2
    random_dup_s_label = np.random.permutation(len(block_s_dup))

    print("After permutation reds-->", random_dup_s_label)

# Function to play the next music file in the shuffled list
def play_s_next_music():
    global current_sound_s_index, sound, random_dup_s_label
    if current_sound_s_index < len(random_dup_s_label):
        sound_file = block_s_dup[random_dup_s_label[current_sound_s_index]]
        current_sound_s_index += 1
        if sound_file != 'Nosound':
            pygame.mixer.music.stop()  # Stop the current music if it's playing
            pygame.mixer.music.load(sound_file)  # Load the new music file
            pygame.mixer.music.play()  # Play the new music
            if sound_file == "tmp_sound/sound_S240.wav":
                sound = "Sound_SCount"
            else:
                sound = os.path.splitext(os.path.basename(sound_file))[0]
            print(f"Now playing red: {sound_file}")
        else:
            sound = "S No sound"
            print("No sound to play red")
    else:
        return
#----------------------------------------------------------------------------------------------------------------------------

color_pic = ""
colors = ["Green", "Green","Red", "Red", "Green", "Green","Green"]
color_label = range(len(colors))

# Initialize
random_label = np.random.permutation(color_label)
#random_label = np.append(random_label,"Green")
color_index = 0
round_num = 0

def stage4(screen): #icon +
    try:
        global duration_time, Group_type, current_sound_index, current_sound1_index,num, check_num,Block,color_pic ,sound
        # ['Subject ID', 'Block', 'No.', 'Picture', 'Sound', 'Result', 'Time']
        global random_label, color_index, round_num,current_sound_p_index,current_sound_s_index, total1, total2, total
        def CSV_result(id_stu, block, choice, picture, sound, result, time):
            t = {'Subject ID':str(id_stu),
                    'Block':str(block),
                    'No.':str(choice),
                    'Picture': picture,
                    'Sound':sound,
                    'Result':str(result),
                    'Time':str(time),
                }
            print(t)
            dict_data.append(t) 
            
        #ดึงไฟล์เสียงมาใช้
        if round_num % 7 == 0:
            while True:
                random_label = np.random.permutation(color_label)
                if colors[random_label[0]] == "Green":  # Ensure the first color is always "Green"
                    break
            print(random_label)
            color_index = 0 

        # Get the current index
        current_index = random_label[color_index]
        selected_color = colors[current_index]
        color_pic = selected_color
        print(selected_color)
        #ถ้าเป็นgroup1 เริ่มใช้เสียงที่ 1...7...  ถ้าเป็นgroup2 เริ่มใช้เสียงที่ 7...1...
        
        # Initialize and randomize the sound files once
        if not block1_dup:
            init_soundp()
        if not block2_dup:
            init_sounds()
        if not block_p_dup:
            init_sound_p()
        if not block_s_dup:
            init_sound_s()
            
        if Group_type == "Group1" :
            if check_num < total1 :
                if color_pic == "Red":
                    play_p_next_music()
                else:
                    playP_next_music()
            elif check_num >= total1 and check_num <= total :
                if color_pic == "Red":
                    play_s_next_music()
                else:
                    playS_next_music()
        else :
            if check_num < total2 :
                if color_pic == "Red":
                    play_s_next_music()
                else:
                    playS_next_music()
            elif check_num >= total2 and check_num <= total:
                if color_pic == "Red":
                    play_p_next_music()
                else:
                    playP_next_music()
                
        button1 = Button((0, 0), (1366, 768), (0,0,0), u"warning".encode('utf8').decode())
        clock = pygame.time.Clock()
        sumer, text = 1, '+'
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        font = pygame.font.SysFont('Sans Serif', 300)
        w = font.render(text, True, (105, 105, 105)).get_width()
        h = font.render(text, True, (105, 105, 105)).get_height()
        #image = pygame.image.load(os.path.join(dirname, 'numberimg\\stage3.png'))
        start_ticks = pygame.time.get_ticks()
        run = True
        while run:

            end_time = pygame.time.get_ticks()/1000
            current_time = (pygame.time.get_ticks() - start_ticks)/1000
            #print("duration_time: ",int(end_time - duration_time))       
            
            if check_num == num:
                print("end")
                stage9(screen)    

            for e in pygame.event.get():
                if e.type == pygame.USEREVENT: 
                    sumer -= 1
                    text = "+" if sumer > 0 else stage5(screen)

                if e.type == pygame.QUIT:
                    #print(dict_data)
                    writecsv()
                    pygame.quit()
                    sys.exit()

                if e.type == pygame.KEYDOWN:
                    if e.key == pygame.K_ESCAPE:
                        writecsv()
                        pygame.quit()  
                        sys.exit()

                if button1.is_clicked(e):
                    # go to stage3
                    #duration_time = start_ticks
                    if Group_type == "Group1" :
                        if check_num <= total1 :
                            if color_pic == "Green":
                                current_sound_index -=1
                            else:
                                current_sound_p_index-=1 
                        elif check_num > total1 :
                            if color_pic == "Green":
                                current_sound1_index -=1
                            else:
                                current_sound_s_index -=1
                    else :
                        if check_num <= total2 :
                            if color_pic == "Green":
                                current_sound1_index -=1
                            else:
                                current_sound_s_index -=1
                        elif check_num > total2 :
                            if color_pic == "Green":
                                current_sound_index -=1
                            else:
                                current_sound_p_index -=1

                    if check_num == 0:
                        CSV_result(id_student,Block, choice_number,"Green", sound, "wrong", '%.3f' %(current_time))
                    else:
                        CSV_result(id_student,Block, choice_number,color_pic, sound, "wrong", '%.3f' %(current_time))
                    print("wrong")
                    stage8(screen)

            screen.fill((0, 0, 0))
            button1.draw(screen)
            #screen.blit(image,(687 - image.get_width(),392 - image.get_height()))
            screen.blit(font.render(text, True, (105, 105, 105)), (683 - (w/2), 372 - (h/2)))
            pygame.display.flip()
            clock.tick(FPS)
        
    except Exception as e:
        print(e)
#--------------------------------------------------------------------------------   

def stage5(screen): #blank
    #if not pygame.mixer.music.get_busy():
        #pygame.mixer.music.load('tmp_sound/generated_sound_1.wav')
        #pygame.mixer.music.play(-1)  # เล่นเสียงแค่ครั้งเดียว
    try:
        global duration_time, current_sound_index, current_sound1_index, Group_type, check_num, num, Set, Set1, add_num, Group_type ,selected_color,Block,color_pic 
        global sound, current_sound_p_index,current_sound_s_index, total, total1, total2
        def CSV_result(id_stu, block, choice, picture, sound, result, time):
            t = {'Subject ID':str(id_stu),
                    'Block':str(block),
                    'No.':str(choice),
                    'Picture': picture,
                    'Sound':sound,
                    'Result':str(result),
                    'Time':str(time),
                }
            print(t)
            dict_data.append(t) 

        button1 = Button((0, 0), (1366, 768), (0,0,0), u"warning".encode('utf8').decode())
        
        clock = pygame.time.Clock()
        sumer, text = 1, u'+'
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        start_ticks = pygame.time.get_ticks()
        run = True
        while run:

            end_time = pygame.time.get_ticks()/1000
            current_time = (pygame.time.get_ticks() - start_ticks)/1000
            #print("duration_time: ",int(end_time - duration_time))
            if check_num == num:
                print("end")
                stage9(screen)   
            for e in pygame.event.get():
                if e.type == pygame.USEREVENT:
                    sumer -= 1
                    text if sumer > 0 else stage6(screen)

                if e.type == pygame.QUIT:
                    #print(dict_data)
                    writecsv()
                    pygame.mixer.quit()
                    pygame.quit()
                    sys.exit()

                if e.type == pygame.KEYDOWN:
                    if e.key == pygame.K_ESCAPE:
                        writecsv()
                        pygame.mixer.quit()
                        pygame.quit()  
                        sys.exit()                   

                if button1.is_clicked(e):
                    # go to stage3
                    #duration_time = start_ticks
                    if Group_type == "Group1" :
                        if check_num <= total1 :
                            if color_pic == "Green":
                                current_sound_index -=1
                            else:
                                current_sound_p_index-=1 
                        elif check_num > total1 :
                            if color_pic == "Green":
                                current_sound1_index -=1
                            else:
                                current_sound_s_index -=1
                    else :
                        if check_num <= total2 :
                            if color_pic == "Green":
                                current_sound1_index -=1
                            else:
                                current_sound_s_index -=1
                        elif check_num > total2 :
                            if color_pic == "Green":
                                current_sound_index -=1
                            else:
                                current_sound_p_index -=1 
                        
                    if check_num == 0:
                        CSV_result(id_student,Block, choice_number,"Green", sound, "wrong", '%.3f' %(current_time))
                    else:
                        CSV_result(id_student,Block, choice_number,color_pic, sound, "wrong", '%.3f' %(current_time))
                    print("wrong")
                    stage8(screen)

            screen.fill((0, 0, 0))
            button1.draw(screen)
            pygame.display.flip()
            clock.tick(FPS)
        
    except Exception as e:
        print(e)
#----------------------------------------------------------------------------------------------------

def stage6(screen): # circle
    try:
        global id_student, choice_number, duration_time, dict_data, check_num, selected_color, double, num,Block,color_pic 
        global random_label, color_index, round_num, sound,current_sound_index,current_sound1_index, total
        
        selected_color = color_pic 
        # Increment the color index and round number
        color_index += 1
        round_num += 1


        def CSV_result(id_stu, block, choice, picture, sound, result, time):
            t = {'Subject ID':str(id_stu),
                    'Block':str(block),
                    'No.':str(choice),
                    'Picture': picture,
                    'Sound':sound,
                    'Result':str(result),
                    'Time':str(time),
                }
            print(t)
            dict_data.append(t) 

        s_rect = screen.get_rect()
        button1 = Button((0, 0), (1366, 768), (0, 0, 0), u"warning".encode('utf8').decode())
        screen.fill((0, 0, 0))
        circle_random = random.randint(2, 6)
        print(circle_random)
        clock = pygame.time.Clock()
        start_ticks = pygame.time.get_ticks()
        running = True
        status = False
        status_circle = True
        time_checkout = 0
        Red = pygame.Color(255, 0, 0)
        Green = pygame.Color(0, 255, 0)
        print(num)
        while running:
            ticks = (pygame.time.get_ticks() - start_ticks) / 1000
            current_time = ticks
            end_time = pygame.time.get_ticks() / 1000
            if check_num == num:
                status = False
                print("end")
                stage9(screen)
                
            if selected_color == "Green":
                if int(current_time) == circle_random:
                    if status == False:
                        time_checkout = ticks
                        status = True
                        check_num += 1
                        
            if selected_color == "Red":
                if int(current_time) == 1:
                    if status == False:
                        time_checkout = 1
                        status = True
                        #check_num += 1
                        
            if selected_color == "Green":
                if int(ticks - time_checkout) == 10:
                    status_circle = False
            #if selected_c#olor == "Red":
            #print(time_checkout,ticks- time_checkout)
            if selected_color == "Red":
                if int(ticks - time_checkout) == 2:
                    status_circle = False

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    writecsv()
                    pygame.quit()
                    sys.exit()

                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        writecsv()
                        pygame.quit()
                        sys.exit()
                else:
                    if status == True:
                        if selected_color == "Red":
                            if status_circle == False:
                                CSV_result(id_student,Block, choice_number,color_pic,sound, "correct", '%.3f' %(ticks - time_checkout) )
                                stage7(screen)
                            else:
                                if event.type == pygame.MOUSEBUTTONDOWN:
                                    x = event.pos[0]
                                    y = event.pos[1]

                                    sqx = (x - (s_rect.left + 684))**2
                                    sqy = (y - (s_rect.top + 384))**2

                                    if math.sqrt(sqx + sqy) < 100:
                                        if event.button == 1:
                                            CSV_result(id_student,Block, choice_number,color_pic, sound, "wrong", '%.3f' %(ticks - time_checkout))
                                            print("wrong")
                                            stagered(screen)
                        else:
                            if status_circle == False:
                                CSV_result(id_student,Block, choice_number,color_pic, sound, "miss", '%.3f' %(ticks - time_checkout))
                                print("miss")
                                stage7(screen)
                            else:
                                if event.type == pygame.MOUSEBUTTONDOWN:
                                    x = event.pos[0]
                                    y = event.pos[1]

                                    sqx = (x - (s_rect.left + 684))**2
                                    sqy = (y - (s_rect.top + 384))**2

                                    if math.sqrt(sqx + sqy) < 100:
                                        if event.button == 1:
                                            CSV_result(id_student,Block, choice_number,color_pic, sound, "correct", '%.3f' %(ticks - time_checkout))
                                            print('correct')
                                            stage7(screen)
                    elif status == False:
                        button1.draw(screen)
                        if button1.is_clicked(event):
                            if check_num == 0:
                                CSV_result(id_student,Block, choice_number,"Green", sound, "wrong", '%.3f' % 0.735)
                            else:
                                CSV_result(id_student,Block, choice_number,color_pic, sound, "wrong", '%.3f' % 0.936)
                            color_index -= 1
                            round_num -= 1
                            print("wrong")
                            stage8(screen)
                            
            if selected_color == "Green":
                color_display = Green 
            else:
                color_display = Red 
            #print("สี : ",color_display)
            if status == True:
                pygame.draw.circle(screen, color_display, (684, 384), 35.8661100)
            pygame.display.flip()
            clock.tick(30)

    except Exception as e:
        print(e)
        
def stage7(screen): #พักก่อนเริ่มข้อใหม่ 2 วิ
    try:
        global choice_number, duration_time, selected_color, num, check_num, total1, total2, Group_type
        clock = pygame.time.Clock()
        sumer, text = 2, u'+'
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        pygame.mixer.music.stop() 
        run = True
        
        print(choice_number, check_num, num)
        while run:
            if Group_type == "Group1" :
                if choice_number == total1 :
                    if selected_color == "Green":
                        choice_number += 1
                        break1(screen)
            else :
                if choice_number == total2 :
                    if selected_color == "Green":
                        choice_number += 1
                        break1(screen)
                
            end_time = pygame.time.get_ticks()/1000
            #print("duration_time: ",int(end_time - duration_time))
            if check_num == num:
                print("end")
                stage9(screen)       
            for e in pygame.event.get():
                if e.type == pygame.USEREVENT: 
                    sumer -= 1
                    if sumer == 0:
                        if selected_color == "Red":
                            stage4(screen)
                        else :
                            choice_number += 1
                            stage4(screen)

                if e.type == pygame.QUIT:
                    #print(dict_data)
                    writecsv()
                    pygame.quit()
                    sys.exit()

                if e.type == pygame.KEYDOWN:
                    if e.key == pygame.K_ESCAPE:
                        writecsv()
                        pygame.quit()  
                        sys.exit()


            screen.fill((0, 0, 0))
            pygame.display.flip()
            clock.tick(FPS)
        
    except Exception as e:
        print(e)
        
def break1(screen): #หน้าพัก
    try:
        global choice_number, duration_time, selected_color,Block 
        clock = pygame.time.Clock()
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        pygame.mixer.music.stop() 
        
        BLACK = (0, 0, 0)
        GREEN = (0, 255, 0)
        WHITE = (255, 255, 255)
        clock = pygame.time.Clock()

        # กำหนดฟอนต์ TH Sarabun New ขนาด 36 พิกเซล
        font = pygame.font.Font("THSarabun.ttf", 36)

        start_time = time.time()  # Record the start time
        countdown_time = 60  # 3 minutes in 180 seconds

        running = True
        while running:
            elapsed_time = time.time() - start_time  # Calculate elapsed time
            remaining_time = max(0, countdown_time - int(elapsed_time))  # Calculate remaining time

            # Convert remaining time to minutes and seconds
            minutes = remaining_time // 60
            seconds = remaining_time % 60
            time_text = f'{minutes:02}:{seconds:02}'

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    writecsv()
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        writecsv()
                        pygame.quit()
                        sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    # Check if the button is clicked
                    if button_rect.collidepoint(event.pos):
                        if elapsed_time >= countdown_time:  # Check if 3 minutes have passed
                            if Block == 1:
                                Block = 2
                            else:
                                Block = 1
                            stage4(screen)  # Proceed to the next stage
                        else:
                            print("Please wait until the break time is over!")
                            
            # ล้างหน้าจอด้วยสีดำ
            screen.fill((0, 0, 0))
            
            # Draw the countdown timer
            timer_text = font.render(time_text, True, WHITE)
            timer_rect = timer_text.get_rect(center=(screen.get_width() // 2, screen.get_height() // 4))
            screen.blit(timer_text, timer_rect)

            # เขียนข้อความ "ทำการทดลองต่อ" ในปุ่ม
            button_text = font.render("ทำการทดลองต่อ", True, BLACK)  # สร้างข้อความ
            button_rect = button_text.get_rect(center=screen.get_rect().center)  # จัดตำแหน่งปุ่มให้อยู่ตรงกลางของหน้าจอ
            pygame.draw.rect(screen, GREEN, button_rect.inflate(20, 20), border_radius=10)  # สร้างปุ่มสีเขียวมุมหม่น
            screen.blit(button_text, button_rect)  # แสดงข้อความภายในปุ่ม
                
            pygame.display.flip()
            clock.tick(FPS)
        
    except Exception as e:
        print(e)

def stage8(screen): #click wrong
    try:
        global duration_time, choice_number, num, check_num
        clock = pygame.time.Clock()
    #     counter, text = 10, u'End'
        counter = 4
        img = pygame.image.load(os.path.join(dirname, 'img/wrong1.png'))
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        font = pygame.font.SysFont('Sans Serif', 40)

        pygame.mixer.music.stop() 
        run = True
        while run:

            end_time = pygame.time.get_ticks()/1000
            #print("duration_time: ",int(end_time - duration_time))
            if check_num == num:
                print("end")
                stage9(screen)
            for e in pygame.event.get():
                if e.type == pygame.USEREVENT: 
                    counter -= 1
                    if counter == 0:
                        #choice_number += 1
                        stage4(screen)

                if e.type == pygame.QUIT:
                    writecsv()
                    pygame.quit()
                    sys.exit()

                if e.type == pygame.KEYDOWN:
                    if e.key == pygame.K_ESCAPE:
                        writecsv()

                        pygame.quit()
                        sys.exit()


            screen.fill((0, 0, 0))
            img_rect = img.get_rect(center=screen.get_rect().center)
            screen.blit(img,img_rect)
            pygame.display.flip()
            clock.tick(FPS)
        
    except Exception as e:
        print(e)
def stagered(screen): #click wrong1
    try:
        global duration_time, choice_number, num, check_num
        clock = pygame.time.Clock()
    #     counter, text = 10, u'End'
        counter = 4
        img = pygame.image.load(os.path.join(dirname, 'img/wrong3.png'))
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        font = pygame.font.SysFont('Sans Serif', 40)

        pygame.mixer.music.stop() 
        run = True
        while run:

            end_time = pygame.time.get_ticks()/1000
            #print("duration_time: ",int(end_time - duration_time))
            if check_num == num:
                print("end")
                stage9(screen)
            for e in pygame.event.get():
                if e.type == pygame.USEREVENT: 
                    counter -= 1
                    if counter == 0:
                        #choice_number += 1
                        stage4(screen)

                if e.type == pygame.QUIT:
                    writecsv()
                    pygame.quit()
                    sys.exit()

                if e.type == pygame.KEYDOWN:
                    if e.key == pygame.K_ESCAPE:
                        writecsv()

                        pygame.quit()
                        sys.exit()


            screen.fill((0, 0, 0))
            img_rect = img.get_rect(center=screen.get_rect().center)
            screen.blit(img,img_rect)
            pygame.display.flip()
            clock.tick(FPS)
        
    except Exception as e:
        print(e)
def stage9(screen): #สิ้นสุดการทดลอง
    pygame.mixer.music.stop()
    try:
        clock = pygame.time.Clock()
    #     counter, text = 10, u'End'
        counter = 2
        img = pygame.image.load(os.path.join(dirname, 'img/end2.png'))
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        font = pygame.font.SysFont('Sans Serif', 40)

        run = True
        while run:
            for e in pygame.event.get():
                if e.type == pygame.USEREVENT: 
                    counter -= 1
                    if counter == 0:
                        writecsv()
                        stage10(screen)

                if e.type == pygame.QUIT:
                    writecsv()
                    pygame.quit()
                    sys.exit()

                elif e.type == pygame.KEYDOWN:
                    if e.key == pygame.K_ESCAPE:
                        writecsv()

                        pygame.quit()
                        sys.exit()

            screen.fill((105, 105, 105))
            screen.blit(img,(210, 327))
            pygame.display.flip()
            clock.tick(FPS)
    except Exception as e:
        print(e)

def stage10(screen):
    try:
        #button1 = Button((177, 327), (400, 100), (65,105,225), u"home".encode('utf8').decode())
        button2 = Button((500, 327), (400, 100), (65,105,225), u"exit".encode('utf8').decode())

        # - mainloop -
        global choice_number, id_sutdent, set_img

        #botton_img1 = pygame.image.load(os.path.join(dirname, 'img/return.png')) #เริ่มการทดลองใหม่
        botton_img2 = pygame.image.load(os.path.join(dirname, 'img/close.png')) #ออกจากการทดลอง

        font = pygame.font.SysFont('Sans Serif', 40)
        clock = pygame.time.Clock()
        start_ticks = pygame.time.get_ticks()
        running = True

        while running:

            # - events -
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        pygame.quit()
                        sys.exit()
                #if button1.is_clicked(event):
                        #id_student = ''
                        #choice_number = 1
                        #stage1(screen)

                if button2.is_clicked(event):
                    # exit
                    #writecsv()
                    pygame.quit()
                    sys.exit()

            # - draws -

            screen.fill((105,105,105))
            #screen.blit(font.render(text, True, (0, 0, 0)), (325, 215))
            #button1.draw(screen)
            button2.draw(screen)
            #screen.blit(img,(235,150))
            #screen.blit(botton_img1,(178,327))
            screen.blit(botton_img2,(500,327))
            pygame.display.flip()

            # - FPS -
            #round_play += 1
            clock.tick(FPS)
            
    except Exception as e:
        print(e)
#--------------------------------------------------------------------
pygame.init()

#screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.FULLSCREEN)
WIDTH, HEIGHT = pygame.display.get_surface().get_size()
pygame.display.set_caption("Login Page")

#screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.FULLSCREEN)
#Str = u"Experiment".encode('utf-8')
#pygame.display.set_caption(Str.decode("utf-8"))
#pygame.mouse.set_cursor((8,8),(0,0),(0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0))

# - start -
stage1(screen)

# - end -
pygame.quit()

Login Clicked
Username: 55555
Cell Type: Group2
new duration time:  7.994
[5 6 0 4 3 2 1]
Green
After permutation --> [20 16 33 19 17 15 28 25 30 24 11  6 29 10  1  3 34 21 26 14 22 13  0  2
  5 32 12  7 27  4 31  9 23 18  8]
After permutation --> [22  5  1 10 19  8  6 34 18 13 28  0 26 12  4 14 29 27 16 31 15 30  9  2
 24 25 11 32  3 17 33  7 23 20 21]
After permutation redp --> [ 7  8 10  9 12  3  4  1  6 11  0  5 13  2]
After permutation reds--> [ 2  7  8  5 12  3  0  4 10 13  9  6  1 11]
Now playing: tmp_sound/sound_S40.wav
2
15
{'Subject ID': '55555', 'Block': '2', 'No.': '1', 'Picture': 'Green', 'Sound': 'sound_S40', 'Result': 'correct', 'Time': '1.174'}
correct
1 1 15
Green
Now playing: tmp_sound/sound_S200.wav
4
15
{'Subject ID': '55555', 'Block': '2', 'No.': '2', 'Picture': 'Green', 'Sound': 'sound_S200', 'Result': 'correct', 'Time': '1.044'}
correct
2 2 15
Green
Now playing: tmp_sound/sound_S40.wav
6
15
{'Subject ID': '55555', 'Block': '2', 'No.': '3', 'Picture': 'Green', 'So

SystemExit: 

C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
